In [ ]:
# Times Square parameters
day = "2025-07-10"

# Weather Forecast: Temperature Outside the Dome

Author: Johnny Esteves

This notebook contains forecast information for the temperature outside the dome. You will find a forecast for a given date and hour of what should be the temperature at the nautical twilight.


The details of each section are explained below. 

In [ ]:
# Standard Library Imports
import warnings
from datetime import UTC, datetime, timedelta

# Third-Party Imports
from astropy.time import Time
from pytz import timezone
from astroplan import Observer

# Local Imports
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

In [ ]:
# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

### Nautical Twilight

Today's information about the nautical twilight.

In [ ]:
# Establish the timeframe for sun set/rise for the night

local_timezone = timezone("America/Santiago")
timestamp_format = "%Y-%m-%d %H:%M:%S %Z"
auxtel_site = Observer.at_site("Rubin AuxTel")

# Datetimes to establish reference points for finding sunset times
# and measurement times for moon illumination
threeAM_local_datetime = datetime.fromisoformat(day).replace(
    hour=3, minute=0, second=0, tzinfo=local_timezone
)

# Astropy Time for astroplan calculations
threeAM_time = Time(threeAM_local_datetime, scale="utc")

# Compute sunrise/sunset
sunset_time = auxtel_site.sun_set_time(threeAM_time, which="next")
sunset_datetime = sunset_time.to_datetime(timezone=UTC)
sunset_local_datetime = sunset_time.to_datetime(timezone=local_timezone)

sunrise_time = auxtel_site.sun_rise_time(threeAM_time, which="next")
sunrise_datetime = sunrise_time.to_datetime(timezone=UTC)

# Nautical sunset and sunrise (preferred by observers over astronomical twilight)
evening_twilight_datetime = auxtel_site.twilight_evening_nautical(
    threeAM_time, which="next"
).to_datetime(timezone=UTC)
evening_twilight_local_datetime = auxtel_site.twilight_evening_nautical(
    threeAM_time, which="next"
).to_datetime(timezone=local_timezone)

evening_twilight = Time(evening_twilight_datetime)

morning_twilight_datetime = auxtel_site.twilight_morning_nautical(
    evening_twilight, which="next"
).to_datetime(timezone=UTC)

In [ ]:
print(5 * "-------")
print("Forecast Temperature of the Day")
print("Date of Forecast: ", day)
print("Civic Twilight Local Time: ", sunset_local_datetime)
print("Nautical Twilight Local Time: ", evening_twilight_local_datetime)
print(5 * "-------")

In [ ]:
# Create an EFD client
client = makeEfdClient()

end_time = Time(evening_twilight + timedelta(hours=6.0))
start_time = Time(end_time - timedelta(days=3.0) - timedelta(hours=6.0))

print(
    f"\nQuery data for {day}"
    f"\n  starts at {start_time.isot} and"
    f"\n  ends at {end_time.isot}\n"
)

In [ ]:
df_outside = getEfdData(
    client=client,
    topic="lsst.sal.ESS.temperature",
    columns=["temperatureItem0", "salIndex", "location"],
    begin=start_time,
    end=end_time,
)

# Select the data from the weather station using the salIndex
mask = df_outside.salIndex == 301
df_outside = df_outside[mask]

# Print the location of this sensor
print("Sensor location: ", df_outside["location"].unique())

# We do not need the salIndex anymore
df_outside = df_outside.drop(columns=["salIndex"])

In [ ]:
# Get the rolling min/mean/max values for the temperature
df_outside = df_outside.rename(columns={"temperatureItem0": "temperature"})
df_outside = df_outside.resample("15min").agg({"temperature": ["min", "mean", "max"]})

df_outside.columns = df_outside.columns.droplevel(0)
df_outside = df_outside.resample("15min").mean()
# df_outside.columns = df_outside.columns.droplevel(0)

In [ ]:
df_outside

## Forecast: Fourier Decomposition Method


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from datetime import timedelta


class FourierFit:
    def __init__(
        self,
        df: pd.DataFrame,
        forecast_time: str,
        ycol="mean",
        frequencies=None,
        window_size_hours=72,
    ):
        """
        Rolling window Fourier fit for time series data.

        Parameters:
        - df: DataFrame containing time series data.
        - forecast_time: Timestamp string marking the forecast start.
        - ycol: Column name of the target variable.
        - frequencies: List of frequencies (in hours) for Fourier terms.
        - window_size_hours: Length of rolling window in hours.
        """
        self.forecast_time = (
            pd.Timestamp(forecast_time).tz_localize("America/Santiago")
            if pd.Timestamp(forecast_time).tzinfo is None
            else pd.Timestamp(forecast_time)
        )
        self.frequencies = frequencies if frequencies is not None else [48, 24, 12]
        self.ycol = ycol
        self.window_size_hours = window_size_hours

        # Only keep last few days for speed (can be tuned)
        self.window_start = self.forecast_time - pd.Timedelta(
            self.window_size_hours, "h"
        )
        self.df = df[df.index >= self.window_start].copy()
        self.params = None

    def _window_slice(self, current_time):
        # window_start = current_time - pd.Timedelta(self.window_size_hours, "h")
        window_start = self.window_start
        return self.df[
            (self.df.index > window_start) & (self.df.index <= current_time)
        ].copy()

    def _fourier_func(self, t, *params):
        # params: [mean, c1, c2, c3, c4, ...freq_coeffs]
        mean = params[0]
        poly_coeffs = params[1:4]  # [c1, c2, c3] for a 2nd degree poly
        trend = poly_coeffs[0] * t + poly_coeffs[1] * t**2 + poly_coeffs[2] * t**3

        f_terms = []
        freq_params = params[4:]
        for i, freq in enumerate(self.frequencies):
            A = freq_params[2 * i]
            B = freq_params[2 * i + 1]
            f_terms.append(
                A * np.cos(2 * np.pi * t / freq) + B * np.sin(2 * np.pi * t / freq)
            )
        return mean + trend + np.sum(f_terms, axis=0)

    def fit_at(self, current_time=None):
        if current_time is None:
            current_time = self.forecast_time
        df_fit = self._window_slice(current_time)
        if len(df_fit) < 2 * len(self.frequencies) + 4:
            raise ValueError(
                "Not enough data points in the current window to fit the model."
            )
        t_fit_hours = (df_fit.index - df_fit.index[0]).total_seconds() / 3600
        y_fit = df_fit[self.ycol].values

        # Initial guess: [mean, c1, c2, c3, A1, B1, A2, B2, ...]
        init_guess = [np.mean(y_fit), 0, 0, 0] + [1, 1] * len(self.frequencies)
        self.params, _ = curve_fit(
            self._fourier_func, t_fit_hours, y_fit, p0=init_guess, maxfev=10000
        )
        self.t0 = df_fit.index[0]  # Save for prediction reference

    def rolling_forecast(self, twilight_time: pd.Timestamp, forecast_issue_times=None):
        """Forecast at each time in predict_times using rolling window fit up to that time."""
        if forecast_issue_times is None:
            forecast_issue_times = pd.date_range(
                self.forecast_time, twilight_time, freq="0.5H"
            )

        # actual temperature at twilight time
        actual = (
            self.df[self.ycol]
            .reindex(self.df.index.union([twilight_time]))
            .interpolate(method="time")
            .loc[twilight_time]
        )
        if twilight_time > self.df.index.max():
            actual = np.nan

        predictions = []
        for issue_time in forecast_issue_times:
            try:
                self.fit_at(issue_time)
                t_twilight, pred = self.predict(twilight_time)
            except Exception:
                pred = np.nan
            predictions.append((issue_time, pred, actual))
        out = pd.DataFrame(
            predictions, columns=["time", "prediction", "actual"]
        ).set_index("time")
        out["prediction"] = np.where(
            out.index > self.df.index.max(), np.nan, out["prediction"]
        )
        out["residual"] = out["prediction"] - out["actual"]
        return out

    def predict(self, timestamp: pd.Timestamp = None):
        if self.params is None:
            raise RuntimeError("Model is not fitted. Call fit_at() first.")
        if timestamp is None:
            timestamp = self.df.index
        if isinstance(timestamp, pd.Timestamp):
            timestamp = pd.Index([timestamp])
        t_hours = (timestamp - self.t0).total_seconds() / 3600
        y_pred = self._fourier_func(t_hours, *self.params)
        return t_hours, y_pred

In [ ]:
data = df_outside.copy()
data["mean"] = (data["max"] + data["min"]) / 2.0
data.index = data.index.tz_convert("America/Santiago")

In [ ]:
start_hour = "06:00"
fourier_fit = FourierFit(data, forecast_time=f"{day} {start_hour}", ycol="mean")
fourier_fit.fit_at()

### Fourier Forecast: Today's Prediction

The input data is sampled every 15min, the prediction have the same frequency. You can check the residual prediction starting from sunrise time up to twilight time.

In [ ]:
import matplotlib.pyplot as plt


class FourierFitPlot(FourierFit):
    def __init__(self, fourier_fit: FourierFit):
        """
        Wrapper class for FourierFit to add plotting capabilities.

        Parameters:
        - fourier_fit: An instance of FourierFit.
        """
        super().__init__(
            fourier_fit.df,
            fourier_fit.forecast_time,
            fourier_fit.ycol,
            fourier_fit.frequencies,
            fourier_fit.window_size_hours,
        )
        self.params = fourier_fit.params
        self.t0 = fourier_fit.t0

    def plot_fit(self):
        # Predict the full fitted series
        time_hours, model_fit = self.predict()

        plt.figure(figsize=(10, 6))
        plt.plot(
            self.df.index,
            self.df[self.ycol],
            "o",
            markersize=3,
            label="Observed Temperature",
        )
        plt.plot(self.df.index, model_fit, "-", color="red", label="Fourier Model Fit")

        for i in range(0, 4):
            plt.axvline(
                pd.Timestamp((sunset_time - timedelta(days=i)).isot).tz_localize(
                    "America/Santiago"
                ),
                color="k",
                ls="--",
            )

        plt.xlabel("Timestamp")
        plt.ylabel("Temperature (°C)")
        plt.title("Temperature Time Series and Fourier Model Fit")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_forecast_residuals(self, forecast_start=None):
        if forecast_start is None:
            forecast_start = (
                self.start_date if hasattr(self, "start_date") else self.forecast_time
            )
        forecast_mask = self.df.index >= forecast_start
        df_forecast = self.df.loc[forecast_mask]
        df_forecast = self.df.loc[forecast_mask]
        # forecast_hours = (df_forecast.index - self.df.index[0]).total_seconds() / 3600
        # forecast_pred = self.model_func(forecast_hours, *self.params)
        forecast_hours, forecast_pred = self.predict(df_forecast.index)
        residuals = df_forecast[self.ycol].values - forecast_pred

        plt.figure(figsize=(10, 4))
        plt.axhline(0, color="gray", linestyle="--", label="Zero Residual")
        plt.scatter(
            df_forecast.index, residuals, color="orange", label="Forecast Residuals"
        )
        plt.xlabel("Timestamp")
        plt.ylabel("Residual (Observed - Model) [°C]")
        plt.title("Forecast Residuals: Observed vs. Fourier Model")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    def plot_forecast_evolution(self):
        forecast_df = self.rolling_forecast(evening_twilight_local_datetime)
        forecast_df["hour_minute"] = forecast_df.index.strftime("%H:%M")
        nautical_index = np.interp(
            evening_twilight_local_datetime.timestamp(),  # Convert nautical time to timestamp
            [ts.timestamp() for ts in forecast_df.index],  # Convert index to timestamps
            range(len(forecast_df)),  # Numeric indices of the dataframe
        )

        # Plot residuals vs hour of the day
        plt.figure(figsize=(10, 4))
        plt.scatter(
            forecast_df["hour_minute"],
            forecast_df["prediction"],
            color="orange",
            label="Fourier Fit Forecast",
        )
        mask = np.isnan(forecast_df["prediction"])
        mean = forecast_df["prediction"][~mask][-1]
        # std = forecast_df["prediction"][~mask][-5:].std()
        plt.axhline(
            forecast_df["prediction"][~mask][-1],
            color="firebrick",
            linestyle="--",
            linewidth=3,
            label="Last Forecast value",
        )  # Add a baseline at y=0

        plt.axhline(
            forecast_df["actual"][-1],
            color="k",
            linewidth=1,
            label="Twilight Measured Temp",
        )

        plt.axvline(nautical_index, color="k", label="Nautical Twilight")

        # Format the x-axis
        plt.xticks(
            range(0, len(forecast_df["hour_minute"]), 2),
            forecast_df["hour_minute"][::2],
            rotation=45,
        )
        # plt.ylim(min(forecast_df["actual"].iloc[0]-1,-0.5+forecast_df["actual"].iloc[0]+forecast_df['residual'].min()),
        #          forecast_df["actual"].iloc[0]+3)
        residual = forecast_df["residual"][~mask]
        plt.ylim(
            max(-1.1 + mean, mean + min(residual) - 0.4),
            max(0.5 + mean, mean + max(residual)),
        )
        # Add labels, title, and legend
        plt.xlabel("Forecast Issue Time [chile local time]", fontsize=14)
        plt.ylabel("Forecasted Temperature at Twilight (°C)", fontsize=11)
        plt.title(
            f"Evolution of Forecasts for Twilight Temperature - {day}", fontsize=14
        )
        plt.legend(fontsize=14, loc=0)

        # Adjust layout and show plot
        plt.tight_layout()
        plt.show()

In [ ]:
pp = FourierFitPlot(fourier_fit)
pp.plot_forecast_evolution()

The figure above displays the prediction for the twilight temperature changes as more weather data accumulates during the day. The forecast at each time uses only data available up to that point. The actual temperature at twilight (grey line) is shown here for reference, but is only known after twilight occurs.

### About the Model

The model is basically three cycles variations of 24, 12 and 8 hours plus a trend, i.e.:

$$
T(t) = Season(t, 24\text{h cycle}) + Season(t, 12\text{h cycle}) + Season(t, \text{h cycle}) + Trend(t) 
$$

where $t$ is the elapsed time in hours since the `forecast_time`. Also the $Season(t, cycle freq)$ is a sum of sin and cos with normalization parameters, for example:
$$
Season(t, 24h) = A_0 \cos(t/24) + B_0 \sin(t/24)
$$

The trend is a second-order polynomial:
$$
Trend(t) = c_0 + c_1 t + c_2 t^2 
$$

The set of free-parameters of this model are nine: ${A_{0/1/2}, B_{0/1/2}\text{ and }C_{0/1/2}}$

### Check The Fitted Model

In [ ]:
pp.plot_fit()
pp.plot_forecast_residuals()